### initial data analysis and final database preparation


In [1]:

import doi as doiLib
from unpywall.utils import UnpywallCredentials
from unpywall import Unpywall
#from prepare_wos_data_to_json import Prepare as prp
import pandas as pd
import json
import seaborn as sns
import numpy as np

In [3]:
with open("email") as fid:
    unpaywallcred = fid.readline()

UnpywallCredentials(unpaywallcred)# could not get api cred on time, so using this email which is pasted all over the web.... #unpaywallcred)

Your email has been set.

In [4]:
dataPath = "../data/derived2/"
dataFile = "wos_data_as.json"


In [5]:
with open(dataPath+dataFile,"r") as fid:
    dataDict = json.load(fid)
#convert dictionary to panda dataframe
data = pd.DataFrame.from_dict(dataDict)


In [20]:
entryType = list()
for item in data["pubType"]:
    if type(item)==str:
        if item == "Article":# or item == "Journal Article":
            entryType.append(True)
        else:
            entryType.append(False)
    else:
        #print(item)
        if "Article" in item:# or "Journal Article" in item:
            entryType.append(True)
        else:
            entryType.append(False)


wosArticles = data[entryType]

dois = list(wosArticles["doi"])
for idx,item in enumerate(dois):
    if item == None:
        dois[idx]="none"

Use unpaywall api to find information about the papers

In [21]:
upwData = Unpywall.doi(dois=dois,errors="ignore",progress=True)


|==================================================| 100%



In [25]:
articles = pd.merge(wosArticles, upwData,on="doi" )
articles.to_json(dataPath+"articles_to_review")


In [19]:
#Open Access

#Paper distribution 
print(articles.oa_status.value_counts(normalize=True))
print(articles.oa_status.value_counts(normalize=False))
#sns.histplot(data = articles.is_oa)



closed    0.384203
gold      0.310576
green     0.168675
bronze    0.074967
hybrid    0.061580
Name: oa_status, dtype: float64
closed    287
gold      232
green     126
bronze     56
hybrid     46
Name: oa_status, dtype: int64


0      http://www.tandfonline.com/doi/abs/10.1080/095...
1                http://www.mdpi.com/1424-8220/8/11/7545
2      https://www.sciencedirect.com/science/article/...
3      https://onlinelibrary.wiley.com/doi/10.1002/st...
4      http://link.springer.com/10.1007/978-90-481-96...
                             ...                        
742    https://www.sciencedirect.com/science/article/...
743    https://www.sciencedirect.com/science/article/...
744           http://www.nature.com/articles/nature24021
745    https://www.triple-c.at/index.php/tripleC/arti...
746              http://www.mdpi.com/1424-8220/17/6/1326
Name: url, Length: 747, dtype: object